Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Deploying a MedNIST Classifier with Ray

This notebook demos the process of deploying a network with Ray Serve as a web service. Ray provides various ways of deploying models with existing platforms like AWS or Azure but we'll focus on local deployment here since researchers are more likely to do this. Ray also provides other libraries for tuning, reinforcement learning, and distributed training in addition to deployment. This tutorial will use MedNIST classifier from the BentoML tutorial so please run at least the training component of that notebook first. The documentation on Ray Serve [start here](https://docs.ray.io/en/master/serve/index.html#rayserve), this notebook will be using the Pytorch specific functionality [discussed here](https://docs.ray.io/en/master/serve/tutorials/pytorch.html).

## Setup environment
To start install the Ray Serve component:

In [ ]:
!pip install "ray[serve]" "protobuf<4"
!python -c "import monai" || pip install -q "monai-weekly[gdown, tqdm]"

## Setup imports
The imports for MONAI are the same as for the BentoML tutorial (assuming it's already installed):

In [3]:
import os
import io
from PIL import Image
import torch
import numpy as np
import requests

import ray
from ray import serve

from monai.apps import download_url
from monai.config import print_config
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    ScaleIntensity,
    EnsureType,
)


print_config()

MONAI version: 1.1.0+11.g7de6c336.dirty
Numpy version: 1.22.2
Pytorch version: 1.13.0+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 7de6c33656a99087ca3b89a817b0879cf093febc
MONAI __file__: /workspace/Code/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.10
Nibabel version: 4.0.2
scikit-image version: 0.19.3
Pillow version: 9.0.1
Tensorboard version: 2.11.0
gdown version: 4.6.0
TorchVision version: 0.14.0+cu117
tqdm version: 4.64.1
lmdb version: 1.3.0
psutil version: 5.9.2
pandas version: 1.1.5
einops version: 0.6.0
transformers version: 4.21.3
mlflow version: 2.0.1
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
resource = "https://drive.google.com/uc?id=1zKRi5FrwEES_J-AUkM7iBJwc__jy6ct6"
dst = os.path.join("..", "bentoml", "classifier.zip")
if not os.path.exists(dst):
    download_url(resource, dst)

This class will represent the service for the model, which accepts an image sent as the body of a POST request and returns the class name in a JSON structure. Note that this class uses asyncio to define the `__call__` to be compatible with the server backend.

In [3]:
MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


@serve.deployment
class MedNISTClassifier:
    def __init__(self):
        # create the transform for normalizing the image data
        self.transform = Compose([EnsureChannelFirst(channel_dim="no_channel"), ScaleIntensity(), EnsureType()])
        # load the network on the CPU for simplicity and in eval mode
        self.net = torch.jit.load("../bentoml/classifier.zip", map_location="cpu").eval()

    async def __call__(self, request):
        image_bytes = await request.body()
        img = Image.open(io.BytesIO(image_bytes))
        img = np.array(img)
        image_tensor = self.transform(img)

        with torch.no_grad():
            outputs = self.net(image_tensor[None].float())

        _, output_classes = outputs.max(dim=1)

        return {"class_index": MEDNIST_CLASSES[output_classes[0]]}

Now the server is started and the classifier backend is associated with an endpoint, which is the route to the service relate to the server address.

In [4]:
serve.start()
MedNISTClassifier.deploy()

2021-10-08 14:49:07,551	INFO services.py:1250 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=2240518) 2021-10-08 14:49:08,881	INFO checkpoint_path.py:15 -- Using RayInternalKVStore for controller checkpoint and recovery.
(pid=2240518) 2021-10-08 14:49:08,886	INFO http_state.py:75 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:ZAewXP:SERVE_PROXY_ACTOR-node:10.246.179.34-0' on node 'node:10.246.179.34-0' listening on '127.0.0.1:8000'
2021-10-08 14:49:09,168	INFO api.py:455 -- Started Serve instance in namespace 'serve'.
2021-10-08 14:49:09,185	INFO api.py:243 -- Updating deployment 'MedNISTClassifier'. component=serve deployment=MedNISTClassifier
(pid=2240517) INFO:     Started server process [2240517]
(pid=2240518) 2021-10-08 14:49:09,214	INFO backend_state.py:896 -- Adding 1 replicas to deployment 'MedNISTClassifier'. component=serve deployment=MedNISTClassifier
2021-10-08 14:49:11,727	INFO api.py:250 -- Deployment 'MedNISTClassifier' is ready at `http://127.0.0.1:80

With the server running in another process we can send it a query with an image and get a response. By default the server will listen on port 8000.

In [5]:
image_bytes = open("./hand.jpg", "rb").read()

resp = requests.post("http://localhost:8000/MedNISTClassifier", data=image_bytes)
print(resp.json())

{'class_index': 'Hand'}


This can also be done on the command line with `curl`:

In [6]:
!curl -X POST "http://localhost:8000/MedNISTClassifier" --data-binary "@hand.jpg"

{
  "class_index": "Hand"
}

Finally shut down the server:

In [10]:
ray.shutdown()

### Command Line Usage

Ray can be started on the command line. Since it operates as a cluster of nodes the first thing to do is create the head node locally then start the serve component:

In [11]:
%%bash

ray start --head
serve start

2021-10-08 14:51:40,274	INFO scripts.py:606 -- Local node IP: 10.246.179.34
2021-10-08 14:51:41,677	SUCC scripts.py:645 -- --------------------
2021-10-08 14:51:41,677	SUCC scripts.py:646 -- Ray runtime started.
2021-10-08 14:51:41,677	SUCC scripts.py:647 -- --------------------
2021-10-08 14:51:41,677	INFO scripts.py:649 -- Next steps
2021-10-08 14:51:41,677	INFO scripts.py:650 -- To connect to this Ray runtime from another node, run
2021-10-08 14:51:41,677	INFO scripts.py:654 --   ray start --address='10.246.179.34:6379' --redis-password='5241590000000000'
2021-10-08 14:51:41,677	INFO scripts.py:659 -- Alternatively, use the following Python code:
2021-10-08 14:51:41,678	INFO scripts.py:662 -- import ray
2021-10-08 14:51:41,678	INFO scripts.py:663 -- ray.init(address='auto', _redis_password='5241590000000000')
2021-10-08 14:51:41,678	INFO scripts.py:671 -- To connect to this Ray runtime from outside of the cluster, for example to
2021-10-08 14:51:41,678	INFO scripts.py:673 -- connect

2021-10-08 14:51:41,553	INFO services.py:1250 -- View the Ray dashboard at http://127.0.0.1:8265
2021-10-08 14:51:42,287	INFO worker.py:826 -- Connecting to existing Ray cluster at address: 10.246.179.34:6379
(pid=2246270) 2021-10-08 14:51:43,070	INFO checkpoint_path.py:15 -- Using RayInternalKVStore for controller checkpoint and recovery.
(pid=2246270) 2021-10-08 14:51:43,102	INFO http_state.py:75 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-node:10.246.179.34-0' on node 'node:10.246.179.34-0' listening on '127.0.0.1:8000'
2021-10-08 14:51:43,781	INFO api.py:455 -- Started detached Serve instance in namespace 'serve'.
(pid=2246314) INFO:     Started server process [2246314]


A separate script with very similar code can then be used to add or replace the backend. This would be useful in an experimental setting where the server is running constantly in the background to which you can push updates quickly as you edit your script.

In [12]:
%%writefile mednist_classifier_start.py

import io
from PIL import Image
import torch
import numpy as np

import ray
from ray import serve

from monai.config import print_config
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    ScaleIntensity,
    EnsureType,
)

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]

ray.init(address="auto", namespace="serve")

@serve.deployment
class MedNISTClassifier:
    def __init__(self):
        self.transform = Compose([EnsureChannelFirst(channel_dim="no_channel"), ScaleIntensity(), EnsureType()])
        self.net = torch.jit.load("../bentoml/classifier.zip", map_location="cpu").eval()

    async def __call__(self, request):
        image_bytes = await request.body()
        img = Image.open(io.BytesIO(image_bytes))
        img = np.array(img)
        image_tensor = self.transform(img)

        with torch.no_grad():
            outputs = self.net(image_tensor[None].float())

        _, output_classes = outputs.max(dim=1)

        return {"class_index": MEDNIST_CLASSES[output_classes[0]]}


MedNISTClassifier.deploy()

# ray.init(address="auto")
# client = serve.connect()

# # remove previous instance of this backend if present
# if "classifier" in client.list_backends():
#     client.delete_endpoint("classifier")
#     client.delete_backend("classifier")

# client.create_backend("classifier", MedNISTClassifier)
# client.create_endpoint("classifier", backend="classifier", route="/image_classify", methods=["POST"])

Overwriting mednist_classifier_start.py


The endpoint is then added by running the script:

In [13]:
%%bash

python mednist_classifier_start.py

2021-10-08 14:52:18,337	INFO worker.py:826 -- Connecting to existing Ray cluster at address: 10.246.179.34:6379
2021-10-08 14:52:18,493	INFO api.py:243 -- Updating deployment 'MedNISTClassifier'. component=serve deployment=MedNISTClassifier
(pid=2246270) 2021-10-08 14:52:18,533	INFO backend_state.py:896 -- Adding 1 replicas to deployment 'MedNISTClassifier'. component=serve deployment=MedNISTClassifier
2021-10-08 14:52:21,277	INFO api.py:250 -- Deployment 'MedNISTClassifier' is ready at `http://127.0.0.1:8000/MedNISTClassifier`. component=serve deployment=MedNISTClassifier


And checked once again for response:

In [14]:
!curl -X POST "http://localhost:8000/MedNISTClassifier" --data-binary "@hand.jpg"

{
  "class_index": "Hand"
}

Finally the service can be stopped:

In [16]:
%%bash

ray stop

2021-10-08 14:52:33,709	INFO scripts.py:861 -- Did not find any active Ray processes.
